In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('data.csv')
data_c = data[0:500]

data_c.select_dtypes(include='number').drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1).corr()
a_none = np.array(None)

,price,spec_rating,display_size,resolution_width,resolution_height,warranty
price,1.000000,0.422260,0.151572,0.624682,0.613066,0.069414
spec_rating,0.422260,1.000000,0.234539,0.291832,0.292554,0.108741
display_size,0.151572,0.234539,1.000000,0.035445,-0.081852,0.033736
resolution_width,0.624682,0.291832,0.035445,1.000000,0.794200,0.017427
resolution_height,0.613066,0.292554,-0.081852,0.794200,1.000000,-0.000770
warranty,0.069414,0.108741,0.033736,0.017427,-0.000770,1.000000


In [4]:
'''Criando uma classe contendo algumas ferramentas para executar o least square'''
class LR():

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.predictions=a_none
        self.predictions=a_none

        
    def least_square(self):
        # Transpose na matrix com as variáveis
        row_x = self.x.T
        dot_x = row_x.dot(self.x)
        invert_dotx = np.linalg.inv(dot_x)
        self.parametros = invert_dotx.dot(row_x).dot(self.y)
        
        return invert_dotx.dot(row_x).dot(self.y)

    def prediction(self, x=a_none):
        if self.parametros.all() == None:
            self.parametros = self.least_square()

        if x.all() == None:
            self.predictions = self.x.dot(self.parametros)
            return self.x.dot(self.parametros)
        else:
            return x.dot(self.parametros)

    def MSE(self, y_hat=a_none, y=a_none): 
        # Quantia de resultados
        if y_hat.all() == None:
            self.predictions = self.prediction(self.x)
            self.mse = (self.predictions - self.y).T.dot((self.predictions - self.y)) / len(self.y)
            return self.mse
        
        else:
            return (y_hat - y).T.dot((y_hat - y)) / len(y)
    
    '''Uma das formas de tentar selecionar os melhores features
    do modelo, assim podemos diminuir a quantia de variáveis'''
    def Forward_selection(self, n_feature):
        k= 1
        # Manter as dimensão do data-set
        axis_x, axis_y = self.x.shape
        # Copiar a data-set pois serão feitas alterações nele
        x = self.x
        # A cada iteração a variável escolhido virá para cá
        select_feature = np.ones([axis_x, n_feature+1])

        for i in range(1, n_feature+1):

            predict = np.inf

            # Mantém a primeira coluna para o intercept e das variáveis já selecionadas
            feature = np.array(select_feature[:,:i+1])
            select = 0
         
            for j in range(1, axis_y-i+1):

                feature[:, i] = x[:, j]
                mse = LR(feature, self.y).MSE(feature, self.y)
                # Encontra a variável que produziu menor mse
                if (mse < predict):
                    predict = mse
                    select = j
            
            # Variável escolhida é adicionada ao data-set das escolhas
            select_feature[:, i] = x[:, select]
            # Variável escolhida é excluída do data-set das buscas
            x = np.delete(x, select, axis=1)

        return select_feature

In [6]:
# Selecionando a coluna da variável e prediction
np_data = data_c[['price', 'spec_rating']].to_numpy()

np_x = np_data.T[1]
np_y = np_data.T[0]

# Adicionando uma coluna com 1 em np_x
np_x = np.array([np.ones(500), np_x]).T

dim_1 = LR(np_x, np_y) 

# Parâmetros calculado pelo least squre
B = dim_1.least_square()
# Prevendo resultado
y_hat = dim_1.prediction(np_x)
# Mean Square Error testando a qualidade do modelo
mse = dim_1.MSE(np_x, np_y)
print(mse)

2457696370.059881
